In [2]:
import os
import shutil
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

# 資料增強與標準化
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [3]:
# 加載數據
train_dataset = datasets.ImageFolder(root='../dataloader_c23/train', transform=transform)
val_dataset = datasets.ImageFolder(root='../dataloader_c23/validation', transform=transform)
test_dataset = datasets.ImageFolder(root='../dataloader_c23/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=2)

In [4]:
from mobilenetv3 import mobilenetv3_large
# 設定設備
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("using {} device.".format(device))

# 加載預訓練的 MobileNetV3 大模型
model = mobilenetv3_large(num_classes=2)
checkpoint = torch.load('mobilenetv3-large-1cd25616.pth', map_location=device)

# 刪除分類層權重
pre_dict = {k: v for k, v in checkpoint.items() if k in model.state_dict() and model.state_dict()[k].numel() == v.numel()}
missing_keys, unexpected_keys = model.load_state_dict(pre_dict, strict=False)

model = model.to(device)

# 定義損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


using cuda device.


In [5]:
# 訓練函數
def train(epoch, epochs, model, train_loader, optimizer, loss_function, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    train_bar = tqdm(train_loader, file=sys.stdout)
    
    for step, data in enumerate(train_bar):
        images, labels = data
        images, labels = images.to(device), labels.to(device)  # 確保數據在正確的設備上
        optimizer.zero_grad()
        logits = model(images)  # 模型輸出
        loss = loss_function(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # 計算準確率
        _, predicted = torch.max(logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1, epochs, loss)
    running_loss = running_loss / len(train_loader)
    accuracy = 100. * correct / total
    return running_loss, accuracy


In [6]:
# 驗證函數
def validate(epoch, epochs, model, validate_loader, loss_function, device):
    model.eval()
    acc = 0.0
    val_loss = 0.0
    val_num = len(validate_loader.dataset)
    
    with torch.no_grad():
        val_bar = tqdm(validate_loader, file=sys.stdout)
        for val_data in val_bar:
            val_images, val_labels = val_data
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            outputs = model(val_images)
            loss = loss_function(outputs, val_labels)
            val_loss += loss.item() * val_images.size(0)

            predict_y = torch.max(outputs, dim=1)[1]
            acc += torch.eq(predict_y, val_labels).sum().item()

            val_bar.desc = "valid epoch[{}/{}]".format(epoch + 1, epochs)
    
    val_loss /= val_num
    val_accurate = acc / val_num
    return val_loss, val_accurate

In [7]:
import time
import sys
from tqdm import tqdm
# 訓練和驗證模型
num_epochs = 25
best_acc = 0.0
train_steps = len(train_loader)
save_path = 'best_mobilenetv3L.pth'
t_l, t_a = [], []
v_l, v_a = [], []
for epoch in range(num_epochs):
    start_time = time.time()
    train_loss, train_accuracy = train(epoch, num_epochs, model, train_loader, optimizer, criterion, device)
    val_loss, val_accurate = validate(epoch, num_epochs, model, val_loader, criterion, device)
    t_l.append(train_loss)
    t_a.append(train_accuracy)
    v_l.append(val_loss)
    v_a.append(val_accurate)
    
    print('[epoch %d] train_loss: %.3f  train_accuracy: %.3f' %
            (epoch + 1, train_loss, train_accuracy))
    print('[epoch %d] val_loss: %.3f  val_accuracy: %.3f' %
            (epoch + 1, val_loss, val_accurate))
    
    if val_accurate > best_acc:
        best_acc = val_accurate
        torch.save(model.state_dict(), save_path)
    end_time = time.time()
    print(f'Training_Time: {end_time - start_time:.2f} seconds')
    
print('訓練完成')

valid epoch[1/25]: 100%|██████████| 55/55 [00:22<00:00,  2.40it/s]
[epoch 1] train_loss: 0.233  train_accuracy: 89.940
[epoch 1] val_loss: 0.347  val_accuracy: 0.877
Training_Time: 118.26 seconds
valid epoch[2/25]: 100%|██████████| 55/55 [00:21<00:00,  2.53it/s]
[epoch 2] train_loss: 0.101  train_accuracy: 96.060
[epoch 2] val_loss: 0.380  val_accuracy: 0.892
Training_Time: 114.83 seconds
valid epoch[3/25]: 100%|██████████| 55/55 [00:21<00:00,  2.56it/s]
[epoch 3] train_loss: 0.071  train_accuracy: 97.231
[epoch 3] val_loss: 0.311  val_accuracy: 0.885
Training_Time: 114.51 seconds
valid epoch[4/25]: 100%|██████████| 55/55 [00:21<00:00,  2.60it/s]
[epoch 4] train_loss: 0.058  train_accuracy: 97.814
[epoch 4] val_loss: 0.401  val_accuracy: 0.890
Training_Time: 115.12 seconds
valid epoch[5/25]: 100%|██████████| 55/55 [00:21<00:00,  2.55it/s]
[epoch 5] train_loss: 0.044  train_accuracy: 98.299
[epoch 5] val_loss: 0.398  val_accuracy: 0.888
Training_Time: 114.09 seconds
valid epoch[6/25]: 1

In [8]:
# 測試模型
model.load_state_dict(torch.load('best_mobilenetv3L.pth'))
model.eval()
test_running_corrects = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        test_running_corrects += torch.sum(preds == labels.data)

test_acc = test_running_corrects.double() / len(test_dataset)
print(f'Test Acc: {test_acc:.4f}')

Test Acc: 0.9102


In [9]:
def save_metrics_to_file(t_l, t_a, v_l, v_a, filename='metrics.txt'):
    with open(filename, 'w') as file:
        file.write("Train Loss:\n")
        for item in t_l:
            file.write(f"{item}\n")
        
        file.write("Train Accuracy:\n")
        for item in t_a:
            file.write(f"{item}\n")
        
        file.write("Validation Loss:\n")
        for item in v_l:
            file.write(f"{item}\n")
        
        file.write("Validation Accuracy:\n")
        for item in v_a:
            file.write(f"{item}\n")

In [10]:
# 假設 t_l, t_a, v_l, v_a 已經被填充
filename = 'mobilenet.txt'
save_metrics_to_file(t_l, t_a, v_l, v_a, filename)

In [ ]:
def load_metrics_from_file(filename='metrics.txt'):
    t_l = []
    t_a = []
    v_l = []
    v_a = []

    with open(filename, 'r') as file:
        lines = file.readlines()
        
        current_list = None
        for line in lines:
            line = line.strip()
            if line == "Train Loss:":
                current_list = t_l
            elif line == "Train Accuracy:":
                current_list = t_a
            elif line == "Validation Loss:":
                current_list = v_l
            elif line == "Validation Accuracy:":
                current_list = v_a
            elif line:
                current_list.append(float(line))
    
    return t_l, t_a, v_l, v_a

In [59]:
# 從文件中讀取
t_l, t_a, v_l, v_a = load_metrics_from_file(filename)

# 確認讀取的數據
print(t_l)
print(t_a)
print(v_l)
print(v_a)

[0.00017941387938076838, 0.00014915590258573905, 0.00014897276882806586, 0.0001344199652537807, 0.0001290054065531068, 0.000123746063941831, 0.00011129486910067498, 0.00011019395043452581, 0.00011555799777852371, 9.799436344635776e-05, 9.115422739543849e-05, 9.921994823445049e-05, 0.00010177693369526727, 9.762458350612886e-05, 9.300677486074467e-05, 9.940537768933508e-05, 8.895165244110911e-05, 8.21552939984637e-05, 8.333765621905008e-05, 7.762585919731969e-05, 7.349414898797274e-05, 8.849507575117362e-05, 8.3043140035847e-05, 7.296231633558313e-05, 7.766158837249451e-05]
[98.27083333333333, 98.58333333333333, 98.5875, 98.70138888888889, 98.76944444444445, 98.8236111111111, 98.96527777777777, 99.00972222222222, 98.91805555555555, 99.13055555555556, 99.16805555555555, 99.09027777777777, 99.05972222222222, 99.12638888888888, 99.19027777777778, 99.06527777777778, 99.18055555555556, 99.22777777777777, 99.2625, 99.33194444444445, 99.31111111111112, 99.18333333333334, 99.22222222222223, 99.3

In [60]:
import torch
from mobilenetv3 import mobilenetv3_large

# 加載預訓練的 MobileNetV3 大模型
model = mobilenetv3_large(num_classes=2)
model.load_state_dict(torch.load('best_mobilenetv3L.pth'))
model.eval()

# 使用 TorchScript 將模型保存為 .pt 文件
example_input = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model, example_input)
torch.jit.save(traced_script_module, 'mobilenet_v3_jit.pt')

In [11]:
test_celeb_dataset = datasets.ImageFolder(root='../CelebDF_v2/extracted_frames', transform=transform)
celeb_loader = DataLoader(test_celeb_dataset, batch_size=256, shuffle=False, num_workers=2)

In [12]:
# 測試模型
model.load_state_dict(torch.load('best_mobilenetv3L.pth'))
model.eval()
test_running_corrects = 0

with torch.no_grad():
    for inputs, labels in celeb_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        test_running_corrects += torch.sum(preds == labels.data)

test_acc = test_running_corrects.double() / len(test_dataset)
print(f'Test Acc of Celeb DF V2: {test_acc:.4f}')

Test Acc of Celeb DF V2: 0.7388
